<h1 style="background-color:black; color:white; padding:10px;">
    Notebook 03 - Dataset do catálogo de peças
</h1>

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Definições iniciais

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Habilitação de recarga de módulo editado.

In [1]:
%load_ext autoreload
%autoreload 2

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Pacotes de uso geral.

In [2]:
import sys
import dtale    # instalado com: conda install dtale -c conda-forge
import numpy as np
import pandas as pd
from html import unescape
from tqdm.notebook import tqdm
from pprint import pprint #, pformat , PrettyPrinter

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Funções do pacote `bricklink_api`.
- Pacote `bricklinp_api`: https://github.com/BrickBytes/bricklink_api .
- Referência da API do Bricklink: https://www.bricklink.com/v3/api.page .

In [3]:
from bricklink_api.auth import DefaultOAuth
from bricklink_api.catalog_item import get_item, Type

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Módulos em diretórios fora de `sys.path` .

In [4]:
# diretório com módulos utilitários
sys.path.append('d:/py_utils')

In [5]:
from pd_utils import (
    d_pd, exemplo_linha, resumo_tipos, resumo_categ, resumo_serie,
    DisplayPandas, ExemploLinha, ResumoTipos, ResumoCateg, ResumoSerie
)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Módulos no diretório do notebook.

In [6]:
from tcc import (
    ofuscar_params, frame2pickle, pickle2frame, object2pickle, pickle2object, coletar_dict)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Utilidades de exibição e resumo de objetos `pandas`.

In [7]:
# primeiro item, 4 items intermediárias, item final e dimensões de objeto pandas
h1s4t1 = DisplayPandas(head=1, sample=4, tail=1)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Outras definições.

In [8]:
d = display
pdod = pd.options.display
pdoc = pd.option_context

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Módulo `tcc`: variáveis comuns a mais de um notebook

In [9]:
# módulo `tcc.py` na pasta do notebok com variáveis comuns aos notebooks
import tcc

In [10]:
# caminhos
d(
      tcc.p_dir_tcc    # diretório inicial TCC
    , tcc.p_dir_bricklink    # diretório dados Bricklink
    , tcc.p_arq_autent    # parâmetros autenticação API Bricklink
    , tcc.pckl_df_catal_bricks    # arquivo pickle dataframe
)

WindowsPath('d:/jup_ws/tcc')

WindowsPath('d:/jup_ws/tcc/bricklink')

WindowsPath('d:/jup_ws/tcc/bricklink/params_autent.json')

WindowsPath('d:/jup_ws/tcc/bricklink/df_catal_bricks.pickle')

In [11]:
# conteúdo ofuscado dos parâmetros de autenticação
ofuscar_params(tcc.params_autent)

{'consumer_key': '********************************',
 'consumer_secret': '********************************',
 'token_secret': '********************************',
 'token_value': '********************************'}

In [12]:
# string com números de partes que comporão o dataset
tcc.txt_nos_bricks

'14716 22886 2356 2357 2453 2453a 2453b 2454 2456 2465 3001 3001f1 3001old 3001oldb 3001oldf1 3001special 3002 3002f1 3002old 3003 3003f1 3003old 3004 3004f1 3004f2 3005 3005f1 3005f2 3005f3 3006 3007 30072 3008 3008f1 3008f2 3009 3009f1 3009f2 3009f3 3010 3010f1 3010f2 3010f3 30144 30145 30400 3065 3066 3067 3245b 3245c 3622 3622f1 3754 3755 4201 4202 4204 46212 49311 6111 6112 6212 6213 700 700e 700eD 700eD2 700eX 702 702old 733 733eX 772 bhol01 bhol02 bhol03 bhol04 bhol05 bhol06 crssprt01 crssprt02 crssprt03 crssprt04 x1214'

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Autenticação para API do Bricklink

- Referência API: https://www.bricklink.com/v3/api.page?page=auth

In [13]:
default_oauth = DefaultOAuth()
default_oauth.set(**tcc.params_autent)

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Catálogo das peças de interesse

- Peças de interesse: https://www.bricklink.com/catalogList.asp?catType=P&catString=5
- Referência API: https://www.bricklink.com/v3/api.page?page=get-item

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Exemplo de resultado da função `get_item` .

In [14]:
pprint(
    get_item(type_=Type.PART, no='2453')
, sort_dicts=False)

{'meta': {'description': 'OK', 'message': 'OK', 'code': 200},
 'data': {'no': '2453',
          'name': 'Brick 1 x 1 x 5 &#40;Undetermined Type&#41;',
          'type': 'PART',
          'category_id': 5,
          'image_url': '//img.bricklink.com/PL/2453.gif',
          'thumbnail_url': '//img.bricklink.com/P/3/2453.gif',
          'weight': '2.13',
          'dim_x': '1.00',
          'dim_y': '1.00',
          'dim_z': '5.00',
          'year_released': 2009,
          'description': '',
          'is_obsolete': False}}


<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Lista dos códigos das peças de interesse.
- Quantidade de peças de interesse.
- Peças de interesse são da categoria *brick*: https://www.bricklink.com/catalogList.asp?catType=P&catString=5

In [15]:
nos_bricks = tcc.txt_nos_bricks.split(' ') 
print(nos_bricks)
len(nos_bricks)

['14716', '22886', '2356', '2357', '2453', '2453a', '2453b', '2454', '2456', '2465', '3001', '3001f1', '3001old', '3001oldb', '3001oldf1', '3001special', '3002', '3002f1', '3002old', '3003', '3003f1', '3003old', '3004', '3004f1', '3004f2', '3005', '3005f1', '3005f2', '3005f3', '3006', '3007', '30072', '3008', '3008f1', '3008f2', '3009', '3009f1', '3009f2', '3009f3', '3010', '3010f1', '3010f2', '3010f3', '30144', '30145', '30400', '3065', '3066', '3067', '3245b', '3245c', '3622', '3622f1', '3754', '3755', '4201', '4202', '4204', '46212', '49311', '6111', '6112', '6212', '6213', '700', '700e', '700eD', '700eD2', '700eX', '702', '702old', '733', '733eX', '772', 'bhol01', 'bhol02', 'bhol03', 'bhol04', 'bhol05', 'bhol06', 'crssprt01', 'crssprt02', 'crssprt03', 'crssprt04', 'x1214']


85

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Coleta dos dados do catálogo para as peças de interesse, em um dicionário, permitindo retomada das consultas somente para códigos de peças ainda não coletados.

In [16]:
# dicionário `dct_guia_precos` armazena resultados de todas consultas realizadas 
# com `get_item` para os códigos de peças em `nos_bricks`
dct_catal_bricks = {}

In [17]:
# função `coletar_dict` só consulta parâmetros ainda não coletados e exibe barra de progresso.
# coletar_dict(dct_coleta=dct_catal_bricks, 
#     iter_tqdm=nos_bricks, desc_tqdm='Importando bricks', unit_tqdm='brick',
#     fn_postfix=lambda no: dict(no=no), fn_chave=lambda no: no, 
#     fn_coleta=lambda no: get_item(type_=Type.PART, no=no)['data']
# )

<img style="float:left;" src="data:image/png;base64, 
iVBORw0KGgoAAAANSUhEUgAAAmgAAAA0CAIAAACxXlO7AAARaUlEQVR4nO2dP4jj3nbHzw15qd6v
l8FTKGyTYnjgIliGbQwb0gSy0rNhpBBYhlTTmemscWFLnZluqzBM8niSYYw0gdeELLhZ8DUJuJni
NUNU2CD1v1flFTfF1ZWubMlj7/x2Z7x7PtWufP+ceyWdc8/3XnsIYwwQBEEQBNmPv3hpAxAEQRDk
mMDAiSAIgiAHgIETQRAEQQ7gL1/agB+I//zf//rdH3//859/fmlDEORgfvrVT//8N//093/9dy9t
CIK8PJhxfjswaiLHy89//vl3f/z9S1uBIK8CDJzfDoyayFGDDzCCcDBwIgiCIMgBbAXOhUuIS1/C
lEOgLiHuYo+CewyHjggZVRQ5jtlAkJchmRiE0/WT7YuEyK9PMjGMSQIAsPZFCcNfi48XrlRLul74
NL1OR6L1LSdARyTtpdrasgKJ362s+EQL2XCkSXh+rVILy51ePnWFectmKfNvu3xdbtXW5MsNllXP
7+xrYOFWPY1bs1cxpTsngfMSGeceZiEI8tpZuDWzMWeMsdgDqyZcavQY6n7MUvqaKB49hg1VAaDu
idWgjDEW+2CdpJE1iZYwnItagVnf6q7jxfz6wm0NnDljjDq2VljX0hFpDSqsXfsXJngrxlYemBcb
/ieZXFjTp8Zb3kLi9yzwYz4JF9vx48tqbZL43Vq5hWvf0Jbeams+J0brwYsZY2zuDFpfNbCJO/sK
WLhEs+X/ikeUPy1y3Kma0sTvWeFT/aBUC9oVY1fa0+UQBJFIoiV0VBUAQGm/1+EhSvjlByhzo3Q2
cNpNgHW0BF2tAwAobw0dltEagIfbN+o+/dJPNgzbGgA0z72OPUszBuoS0npwnE6FtZ+DsGO06wB1
83IYBp/lQEJvTNArKsLC5WlWeQvrWTDVjbcKgGL2nPB+thGgvqxWgbVvkJp16jiln9bNQKwzNuYT
Tvlt0NpDCB+jHT2UtrY34s6+NHREiLZ0hnp+qdnPl27NtgMhn5xdU7q4sUDXt68XqQ6cPC+cuJnI
sCk4bBUoDKAgESR+l7gjlxBCRv/qnlghhNYJD/6J391UdeiIkJEr1IcyMWc0k6cr0ymqxNtIWJ4W
WPsGcd1RekXWH7bFDUHid4WoUiKMUBdzaOQHQ1EbMI0iAIBkdh/q79sKAEAUTdO4WGAxs3m0q6sN
4b/yoFIZbrdJogeQQ+wySj1PmzJ2d16MvfmLKQUSACgEEjpqAf1obHfF3/RPbb6wLm9hHYXQyMcr
JiTTAA+pVYV6uWLsqr2rCAAU5hPUN9lShs4G4LzLM3/hckWyvnBJ13W7hBDD/x9ZDqx2rWvfyHx+
dmdzBX73Plrid4kxckXhgphPivJ1UfYXnnnhSl1QN7Pk3Zyx4PzNk/ME1VNKXQ3m1yXPwga7M87Q
emzzJDc0axfwkTEW+3po3tCswL0aM65C1PhIqiQC+0GNGWNX/9JfeTro3iow65BMLqxTnknHXsce
Z9F3sFRXjDE2H4ZWj08idTXb4QrPm6XIxhO/21pyXWgzE8+wrcfLrQL28k3MGOtLCyXJ8th7kMWN
xO/WgvcxuzMVoK4QRhhtCNu0/sHLNAQ5cpp9tlLHhBBSi3osOFMAeEIZWiebDjSJliLaaX0Wq9eE
EFJ7vGR3pgi3YGtiQfqUrihCrKKeZtc0rSTpKbyYWbhV30gZxdofP3jnG3WzkMkKclR5C2naDVBX
xVXFvMt9y961Kqhr2pO+Ze0bhNTM0OmZ6dScBawX1QghZKyuJEc3sKIed612K9sLnNrQY4wFZi1r
UXKtK28pu9a1b5wExiq94+LOJn4vVeDZyltqTxwNCQdwKdx+i2cpZcq/chawDa40aPYzlZ6OWvZw
zi3RmruEw2QytkFkxhVTmkzGS/98H/lxd+DUvQ8aAL+1XFgARW3IJdL7VDcvh2B/osW1p3bu65kK
IS4WUM4C8VxGkSw3Dy/NbN3EWcyyYStnl2mKnSseXLfZUGCKoygUELVyZMsV8044AliOu7XgfSz+
CwChdUsBAJp98dojyA9HMjHISXTJGGOs/UmINOsoBF2sLLOlajK7h/SNW/sGqXHfzd7N0oRjHS0B
HOF2s1X4NxiE3wuM68JbnEwMcs1X+cezg1M3gzRipYsVOiLpKNhH6ElrkU66StA+ePo0mKXhcEtr
lV2rrN8+jI2TwFhlyxHpzgLY136Slu/vnjs9jU/50idX4EExew4MZrtCb7M/H9otQloDZ77PbVq4
NTN06E6r1v7FvfHxbC+P/sw9zlyTEREuiqZ7Si4AIMue42XVHgMAFDZUAEBVeeENxaOcrIC8Pt2m
yvIQQE6ytT6bO4NWhaKLID8Iyew+FO4PtA+ePhj7a76rJLxqvlSNomn6Giafg1D4br5JOZ4k3O+n
WVG+Cq9EyLNJ9HCAxZk8Gz2Kwx+LG+v0ckMrynO1rbe7pAWQhNZ1VHqo5MtqfQn51NHZQGQ1fBs1
82CZblwv5kAbVLnWKUBHZA4A0p1VzLvYA6tW1Fr3ZlOBT6+WSrUAwB85gCdiIWfhEs12aEFf3Ibe
Wo3evonQMwOn2GvNnwlV7eS7Dk+R+Ne2OIBXtscgIW2oQJ6e1tVsM3yf7na+ZlWWO5d3wXwo9AQA
AK3P0qWxPmh9q6UxghwlDVXJt8H2pvqg0Obyd89lekEmFe3TTzaki+CaNYXQrKV+udkXCXHuqUtb
2BRa88X9rn6frPWllIefctbRcsenVa51eBnczZ3M7xXurGLecc84d6b7HBWWUdTTkhNM5VItQHos
uaPbT2nCsHCJtvRWT0RNvtRINwvSUzi7dmqfe6o2XRiu/XG6/6y032fyLL0xw1KFViadrMXNE8fB
m20HxCboYpbucdbbRqa+Lm6s6bYAC5DXqirAkS3f/MaSdiWeFfm7NHW1AWXnIBDk+0dpv8+VGHpr
8WMpycSQTnZcWFOn3QT6yc4OpyhvDX1q3aRuV7yS8hemF25rsOM9Be2dyJ8WN7z9vcx9a6Q58dof
i/a1q8wfx14HdD8uyLNZ+OT7bWUtQL1tpIczEv/a3nZ3X1brAGSPlLs4rT0Uwikk/nWmggJwY6Rb
Vo7sWje/N6/1sy3G/M7KZVS1c4juyBt9l8mzRYPLSCYXFngfN1OaLda+odkO3ecAikiHeEYE+u5Y
+8wfedcdGBMSAoDux/10AzKYP5IasQAAhnN2pgAUlx71ttGxrBMSUdbvOZbWIgOA4Xw+hNZjBFA1
3Vp/5RknNWICdLJD54p5N49IjZgAAA5lZRPkGHDBjUwLVGSoBcs7XnynwSI7vqud+3pNc9usH9CI
nPBCoPtxvw4A1CVjdYXng5AfCP6+tIgNwN8XUwGAsyAGoybeD2/V1yDxH3T1g6hWNwMaEY1XSw8J
Qr3PqEsI4UUqXmRBsz8fpv06lO1MZKUXs25+9IPaCbEAdD8+4FVt9pnQkMtaUMxrL0j9khenO2SJ
361FPdZvfmmtJ1m45FqN70yFz6fwSNnUaVex163lfjhbEAwN6BEyBQBnznYokwXXqvtx0AT6KZuT
c69Ta41+4+V3lvtncQuHc9Y8cETNfuwbNd5Ax4urdy6TiVEzwVuZCoByNXdIy3gjn0GRSn4OQoAw
fdjEQPbbxRSU+3bCGDukFYn0bBVGi335h/94/9ImIMiz+MM/3r9Mx1mceJnuvxUL14X+XmHmWDj+
ESUTd/a2vxHm8AcQEARBXgX0E7yGXxL4BTn+ESWzR3Vb0MbAiSDIMTC1at/7z4xoV3ucET0qjn9E
inlVonM8Q6pFDgSlWuTYeTGpFkFeE5hxfjt++tVPL20Cgnw5+AAjCOeZp2qRA/jtif5v//3vf/q/
P720IQhyML/+q1//9m+f/O1rBPkhQKkWQRAEQQ4ApVoEQRAEOQAMnAiCIAhyABg4EQRBEOQAMHAi
CIIgyAFg4EQQBEGQA8DAiSAIgiAHgIETQRAEQQ4AAyeCIAiCHAAGTgRBEAQ5AAycCIIgCHIAGDgR
BEEQ5AAqAycdEWOS5P9f+wYhhBDS9aWr1OUXiUu3SubV174hF0AQ5Ltk7Ru/+J/M5P6k6ydA3a/r
RhK/W3R6r4inbaOjDef8VC3hqN3FL2fmD0N54KQj0hrIFxK/Z4EfMxZ7YF2IO0FHLXs4Z4zNh3Zr
RLOSDcoYmzfMC/4K0VurQY/9z5kiCLI/1P2FPDK9tRqUsV5UIy172H5BN5JMjK3I9GpY++MHL74r
+ZPLldTNgMVe5+vZ9D2zHTipS0jrwXHkCV3PgqluvFUAFLPnhPezBACAzgbgvNMAQPvg6YOZWAzq
al2u648fvPPmVxsBgiCvhLoZsMCsP11wf7Qr1m8CNPuMMXaFy+8K6mZwUNREnkdJxtmmjN2dq/Kl
dRRCIw+H0ygCgHW0LMTIZZRKNKH4R0OtA721Gj28owjy3ZFt32TJZSrVJn63ZQPYGnEXQEeEpHJU
MWlbiH0eeVsn3/3JE1Y6ygvmOnDeu7goNUiK+0rJxNi4UtqRTN4pN37h1swQplaNtyCNPRvdRgPb
7W+bQUeEjFxRcufoRBuy+prP7cLNh5Zvq91ExYqZRbhx9ky2A6emlWaHHTUNpXVV+mu2IprW1UZ6
RTGvvaVGCGkt/XMN000E+T7JNmUYW3lLTfbFink3dwAcyvqV7z51taW3YowxRp3QvKG8zW5r6cei
TcNfAyzc1oMXM8YYmw9D65am1U/S3mMfrBOXgshKZXgStnBrZmPOWzi1aiNa6Ig6tra5L5tMDNFp
7D20jEkCzX7s69Dhcmhh7PpgvLWtWz4QYUbsATcDAAAGS3UlRtfjwS8fHaONdHSH3BfgXffAnmYj
urBO+RzEXscev9Kt3KPhK5yqrZsBY4yx4ExJ002xEsRdaAT5nrCv/QT4K3/oIQatL0TdJFqm1/It
IUn1bfbT+AdJ9CBqL2Y2OO0mAIBydumAPav2LfSTDWJzVLti7EordNQ89zph8FkOJMnsPnRSnUze
nMpQzDuxJlhH4XaXZQORzFDMngPZ3tbwks+D+kakJNLooNlnB83t5tCExWeBELqjaFpVGdmXvQMn
l2dh40ER8uw6e/Yl0nQz8a+X3mp7WYogyPGimHexB1at5LD9PuTK4cW9uLaxJZRezETLiyCrHC1z
DSyjXKpNogfQ3xTLlnaUE0VTsDXRiGbn3k+QC7nXS327gZL2y8yooHx0e1LoWlFPxfV8csZLPBD0
bPYLnAV5Vsi2uTzL2XwQxe6mWODU1Ua+D4ogyLGjmHdcEp070/yw/V4sbqypw6XDoCccSV3Vt1wE
vbXCYVrwUoQBRW1sB7MKqVZRTyF8LJYt60hCVTvgULmhYs639scDPdWZr42SBkraLzOjgvLR7Umh
6yxHT/xrW+f6LftYZjFyGHsGzraRyuKJf23r79sKAIDWFlsO9NYKN06K57ubqsoXOOtouWuVhyDI
ESF/4URVO9BQy48Aqm90IUsms3tZ1+T+PfGv7fRCvW3kqqnU/kPEz+OMs+/INduZPJtMxrmwWYb2
LtdF09NJckeLGyvTNlOU9ns9VaHLvx8J2RFIemuVSLVlA5HMSPzrXD0uQRrd1ldjFfUU5O81lHad
7gSvZ4GkyqZhe3FjoVT7fDbXaCmx1wGxQhF74LxCx5Ouzp20mXTxmH8wlJZsNC1VXMQhCHLMZD4B
AHhSuPJ0SFOx+RAAuA/JvITu+Y5wINk3CHWP5rWkwsL/SJ5nzo/nbPae1a0k9jNLM0+VdyT8UsHp
cfsLVdIenbn0qUPTY1BbbA1ENqOTHXcSU8c/LRkdb1yyLf/I8Xw9rU4dyTPnbtkZilrCCcNwLnUa
e5u5NbIXhDFWHVURBEG+LWvfuFUD/MrmtyDxu7Wot+PwM1IO/lYtgiCviboZvJtVfDkSQV4FGDgR
BHk9UJcQotn8J8mQr8jaN0gN9zu/DJRqEQRBEOQAMONEEARBkAPAwIkgCIIgB/D/ub8r+gux2M8A
AAAASUVORK5CYII=
" />

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Carga de `dct_catal_bricks` a partir de arquivo pickle, ao invés da coleta online.

In [19]:
# depois da importação inicial, dicionário foi salvo em arquivo pickle e pode
#  ser carregado desse arquivo a qualquer tempo sem necessidade de nova coleta.
dct_catal_bricks = pickle2object(tcc.pckl_dct_catal_bricks)

Lido: d:\jup_ws\tcc\bricklink\dct_catal_bricks.pickle
Tamanho: 28325 bytes
Modifificado em: 31/08/2021 10:51:37


<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Dataframe `df_catal_bricks` com informações do catálogo das peças de interesse.

In [20]:
df_catal_bricks = (
    pd.DataFrame.from_records(list(dct_catal_bricks.values()))
    .assign(name=lambda df: df.name.apply(unescape))    # substituir entidades html
    .set_index('no')    # código da peça como índice
)

In [21]:
# criação de colunas derivadas do código da peça, para faciltar classificação:
#     - `pfx_no`: números na parte inicial do código;
#     - `sfx_no`: demais caracteres seguintes.
pfx_sfx_no = ['no_pfx','no_sfx']
df_catal_bricks = df_catal_bricks.drop(columns=pfx_sfx_no, errors='ignore').join(
    df_catal_bricks
    .index
    .to_series()
    .str.extract(r'^(\d*)(.*)$')
    .set_axis(pfx_sfx_no, axis=1)
    .assign(no_pfx=lambda df: df.no_pfx.replace('',0))
    .astype({'no_pfx':int})
).sort_values(by=pfx_sfx_no)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Amostra e dimensões do dataframe.

In [22]:
with pdoc('max_colwidth', 20):
    h1s4t1(df_catal_bricks)

,name,type,category_id,image_url,thumbnail_url,weight,dim_x,dim_y,dim_z,year_released,is_obsolete,alternate_no,description,no_pfx,no_sfx
no,,,,,,,,,,,,,,,
bhol01,Brick 4 x 4 Corn...,PART,5,//img.bricklink....,//img.bricklink....,3.50,4.00,4.00,1.00,0,False,NaN,Earlier versions...,0,bhol01
crssprt02,Brick 1 x 6 with...,PART,5,//img.bricklink....,//img.bricklink....,2.52,1.00,6.00,1.00,1955,False,NaN,NaN,0,crssprt02
3004,Brick 1 x 2,PART,5,//img.bricklink....,//img.bricklink....,0.83,1.00,2.00,1.00,1959,False,93792,This part has at...,3004,
3008f1,Brick 1 x 8 Tran...,PART,5,//img.bricklink....,//img.bricklink....,3.21,1.00,8.00,1.00,0,False,NaN,NaN,3008,f1
3066,Brick 1 x 4 with...,PART,5,//img.bricklink....,//img.bricklink....,1.71,1.00,4.00,1.00,1979,False,35256,NaN,3066,
49311,Brick 1 x 4 x 3,PART,5,//img.bricklink....,//img.bricklink....,3.97,1.00,4.00,3.00,2019,False,NaN,NaN,49311,


(85, 15)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Salvamento e teste da carga do dataframe em formato pickle.

In [23]:
frame2pickle(frame=df_catal_bricks, p_pickle=tcc.pckl_df_catal_bricks)

Salvo: d:\jup_ws\tcc\bricklink\df_catal_bricks.pickle
Tamanho: 20308 bytes
Modifificado em: 27/10/2021 13:33:11


In [24]:
_ = pickle2frame(p_pickle=tcc.pckl_df_catal_bricks)
del _

Lido: d:\jup_ws\tcc\bricklink\df_catal_bricks.pickle
Tamanho: 20308 bytes
Modifificado em: 27/10/2021 13:33:11
